In [1]:
# This script is for multicollinearity diagnosis

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import math
import os
from scipy import stats

In [28]:
# Standardization function, one of the most commonly used standardization patterns
def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

In [29]:
# Data Preparation
data = pd.read_csv("../Data/03 Data for Multicollinearity Diagnosis/Mul-Diagnosis.csv")  # You can use relative path ../ here
data = data.set_index("GEOID")

In [30]:
data.head()

,obesity_cr,% White,% Black,% Ame Indi and AK Native,% Asian,% Nati Hawa and Paci Island,% Hispanic or Latino,% male,% married,% age 18-29,...,% >=highschool <university,% >=university,median income,% unemployment,% below poverty line,% food stamp/SNAP,median value units built,median year units built,% renter-occupied housing units,population density
GEOID,,,,,,,,,,,,,,,,,,,,,
36081046200,26.6,0.035653,0.128637,0.001426,0.405733,0.000000,0.324729,0.429122,0.443847,0.207074,...,0.499685,0.261490,53295,0.056929,0.086140,0.167808,566100,1964,0.776419,28106.912090
36081045000,19.6,0.123575,0.078431,0.000000,0.579115,0.000000,0.191518,0.447788,0.534483,0.176015,...,0.524725,0.367445,90568,0.077815,0.071591,0.076923,585500,1939,0.326454,12744.813160
36081045400,21.5,0.056357,0.160195,0.008653,0.628800,0.000000,0.101176,0.526514,0.550474,0.151986,...,0.485987,0.354435,45958,0.064898,0.147992,0.089735,168400,1954,0.558124,19521.893370
36081045600,18.6,0.042360,0.028744,0.000000,0.708775,0.000000,0.142209,0.486384,0.604982,0.216339,...,0.438776,0.380612,76563,0.081081,0.170197,0.201201,650900,1939,0.279279,8153.246497
36081044602,30.6,0.020073,0.197928,0.011224,0.427585,0.016836,0.298295,0.462335,0.505800,0.136844,...,0.497857,0.267062,40883,0.089128,0.325225,0.340757,281000,1957,0.869339,27255.666740


In [31]:
len(data)

3019

In [32]:
#separate the variables to predictors and true label
X_sociodemo = data.copy()
y = X_sociodemo.pop('obesity_cr')

In [33]:
#standardization
X_sociodemo_stats = X_sociodemo.describe().transpose()
X_sociodemo_stats
X_sociodemo_std = standarize_data(X_sociodemo, X_sociodemo_stats)

In [34]:
X_sociodemo_std.head()

,% White,% Black,% Ame Indi and AK Native,% Asian,% Nati Hawa and Paci Island,% Hispanic or Latino,% male,% married,% age 18-29,% age 30-39,...,% >=highschool <university,% >=university,median income,% unemployment,% below poverty line,% food stamp/SNAP,median value units built,median year units built,% renter-occupied housing units,population density
GEOID,,,,,,,,,,,,,,,,,,,,,
36081046200,-1.009433,-0.264057,-0.326255,1.738132,-0.229260,0.006933,-1.367030,0.058533,0.331331,-0.142422,...,0.327751,-0.412378,-0.413220,-0.322810,-0.794741,0.029271,-0.186505,0.785947,0.685887,1.091729
36081045000,-0.702857,-0.449446,-0.449616,2.854605,-0.229260,-0.501207,-0.905631,0.885515,-0.102434,-0.439707,...,0.511843,0.092906,0.691226,0.237599,-0.920909,-0.631793,-0.129471,-0.945095,-1.140317,-0.141482
36081045400,-0.937241,-0.147522,0.298891,3.174540,-0.229260,-0.845824,1.040279,1.031425,-0.438016,-0.403113,...,0.227047,0.030861,-0.630625,-0.108984,-0.258324,-0.538605,-1.355699,0.093530,-0.200075,0.402556
36081045600,-0.986046,-0.632921,-0.449616,3.689528,-0.229260,-0.689302,0.048363,1.528773,0.460728,-0.748845,...,-0.120050,0.155698,0.276240,0.325248,-0.065754,0.272159,0.062797,-0.945095,-1.331778,-0.510076
36081044602,-1.063758,-0.008190,0.521251,1.878843,4.117651,-0.093902,-0.546065,0.623810,-0.649479,0.602024,...,0.314309,-0.385805,-0.781004,0.541177,1.278735,1.287237,-1.024667,0.301255,1.063009,1.023395


In [35]:
y.head()

GEOID
36081046200    26.6
36081045000    19.6
36081045400    21.5
36081045600    18.6
36081044602    30.6
Name: obesity_cr, dtype: float64

In [36]:
# VIF test
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [45]:
# First test
# VIF dataframe
X = X_sociodemo_std.copy() 
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
# First test result
vif_data.astype(str)
vif_data_1 = vif_data.round(3)

C:\Users\ryanz\anaconda3\envs\Obesity\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [46]:
vif_data_1

,feature,VIF
0,% White,58.265
1,% Black,49.264
2,% Ame Indi and AK Native,1.091
3,% Asian,19.277
4,% Nati Hawa and Paci Island,1.035
5,% Hispanic or Latino,44.031
6,% male,1.333
7,% married,4.260
8,% age 18-29,4.314
9,% age 30-39,3.402


In [47]:
# # Second test; continue
# VIF dataframe
X2 = X_sociodemo_std.drop(columns=['% >=highschool <university'])
vif_data2 = pd.DataFrame()
vif_data2["feature"] = X2.columns
  
# calculating VIF for each feature
vif_data2["VIF"] = [variance_inflation_factor(X2.values, i)
                          for i in range(len(X2.columns))]

vif_data2_1 = vif_data2.round(3) 

In [48]:
vif_data2_1

,feature,VIF
0,% White,58.261
1,% Black,49.260
2,% Ame Indi and AK Native,1.091
3,% Asian,19.273
4,% Nati Hawa and Paci Island,1.032
5,% Hispanic or Latino,44.031
6,% male,1.332
7,% married,4.260
8,% age 18-29,4.314
9,% age 30-39,3.401


In [49]:
# Final variables
# variables to be removed
remove_vars = ['% White','% >=highschool <university','% >=university']
X3 = X_sociodemo_std.drop(columns=remove_vars)
vif_data3 = pd.DataFrame()
vif_data3["feature"] = X3.columns
vif_data3["VIF"] = [variance_inflation_factor(X3.values, i)
                          for i in range(len(X3.columns))]
vif_data3_1 = vif_data3.round(3)

In [50]:
vif_data3_1

,feature,VIF
0,% Black,2.754
1,% Ame Indi and AK Native,1.060
2,% Asian,2.026
3,% Nati Hawa and Paci Island,1.020
4,% Hispanic or Latino,4.729
5,% male,1.317
6,% married,4.133
7,% age 18-29,4.216
8,% age 30-39,3.219
9,% age 40-49,1.765


In [43]:
# One more test variables
# variables to be removed
remove_vars = ['% White','% >=highschool <university','% >=university','% food stamp/SNAP']
X4 = X_sociodemo_std.drop(columns=remove_vars)
vif_data4 = pd.DataFrame()
vif_data4["feature"] = X4.columns
vif_data4["VIF"] = [variance_inflation_factor(X4.values, i)
                          for i in range(len(X4.columns))]
vif_data4_1 = vif_data4.round(4)

In [44]:
vif_data4_1

,feature,VIF
0,% Black,2.7008
1,% Ame Indi and AK Native,1.0602
2,% Asian,2.0149
3,% Nati Hawa and Paci Island,1.0197
4,% Hispanic or Latino,4.6762
5,% male,1.3167
6,% married,4.1303
7,% age 18-29,3.9272
8,% age 30-39,3.0640
9,% age 40-49,1.6909
